In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv
/kaggle/input/crime-cast-forecasting-crime-categories/train.csv
/kaggle/input/crime-cast-forecasting-crime-categories/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/crime-cast-forecasting-crime-categories/train.csv")
test = pd.read_csv("/kaggle/input/crime-cast-forecasting-crime-categories/test.csv")

In [3]:
train.shape, test.shape

((20000, 22), (5000, 21))

In [4]:
replacement_dict = {'Property Crimes': 0,
                    'Violent Crimes': 1,
                    'Crimes against Public Order': 2,
                    'Fraud and White-Collar Crimes': 3,
                    'Crimes against Persons': 4,  # Add placeholders for missing values (if needed)
                    'Other Crimes': 5} 
train['Crime_Category'] = train['Crime_Category'].replace(replacement_dict)


/tmp/ipykernel_33/4284866162.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Crime_Category'] = train['Crime_Category'].replace(replacement_dict)


In [5]:
# Convert the date columns to datetime format
train['Date_Reported'] = pd.to_datetime(train['Date_Reported'])
train['Date_Occurred'] = pd.to_datetime(train['Date_Occurred'])

# Calculate the difference in minutes
train['time_between_date_occured_and_reported'] = (train['Date_Reported'] - train['Date_Occurred']) / pd.Timedelta(minutes=1)

# Get the absolute difference in minutes
train['time_between_date_occured_and_reported'] = train['time_between_date_occured_and_reported'].abs().astype(int)

# Print the dataframe
print(train)

                                       Location  \
0       4500    CARPENTER                    AV   
1               45TH                         ST   
2        600 E  MARTIN LUTHER KING JR        BL   
3      14900    ORO GRANDE                   ST   
4       7100 S  VERMONT                      AV   
...                                         ...   
19995   5100 W  ADAMS                        BL   
19996  16900    ROSCOE                       BL   
19997   1000 S  SHENANDOAH                   ST   
19998    300 W  SEPULVEDA                    ST   
19999           DALTON                       AV   

                          Cross_Street  Latitude  Longitude Date_Reported  \
0                                  NaN   34.1522  -118.3910    2020-03-09   
1      ALAMEDA                      ST   34.0028  -118.2391    2020-02-27   
2                                  NaN   34.0111  -118.2653    2020-08-21   
3                                  NaN   34.2953  -118.4590    2020-11-08   
4 

/tmp/ipykernel_33/1605677976.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['Date_Reported'] = pd.to_datetime(train['Date_Reported'])
/tmp/ipykernel_33/1605677976.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['Date_Occurred'] = pd.to_datetime(train['Date_Occurred'])


In [6]:
# Convert the date columns to datetime format
test['Date_Reported'] = pd.to_datetime(test['Date_Reported'])
test['Date_Occurred'] = pd.to_datetime(test['Date_Occurred'])

# Calculate the difference in minutes
test['time_between_date_occured_and_reported'] = (test['Date_Reported'] - test['Date_Occurred']) / pd.Timedelta(minutes=1)

# Get the absolute difference in minutes
test['time_between_date_occured_and_reported'] = test['time_between_date_occured_and_reported'].abs().astype(int)

# Print the dataframe
print(test)

/tmp/ipykernel_33/3883583963.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test['Date_Reported'] = pd.to_datetime(test['Date_Reported'])


                                      Location  \
0      1500    LEIGHTON                     AV   
1       100 S  NORMANDIE                    AV   
2       300 E  111TH                        ST   
3      1300 S  LA BREA                      AV   
4     11000    MORRISON                     ST   
...                                        ...   
4995   4600    MASCOT                       ST   
4996   2200 E  7TH                          ST   
4997           LANGDON                      AV   
4998    400 E  5TH                          ST   
4999  10100 S  SAN PEDRO                    ST   

                         Cross_Street  Latitude  Longitude Date_Reported  \
0                                 NaN   34.0128  -118.3045    2020-03-03   
1                                 NaN   34.0726  -118.3029    2020-06-01   
2                                 NaN   33.9348  -118.2695    2020-08-28   
3                                 NaN   34.0497  -118.3442    2020-12-23   
4                  

/tmp/ipykernel_33/3883583963.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test['Date_Occurred'] = pd.to_datetime(test['Date_Occurred'])


In [7]:
X_train = train.drop(['Location', 'Date_Reported',
       'Date_Occurred', 'Area_Name',
       'Modus_Operandi',
        'Premise_Description',
       'Weapon_Description', 'Status_Description','Crime_Category','Cross_Street'], axis=1)
X_test = test.drop(['Location', 'Date_Reported',
       'Date_Occurred', 'Area_Name',
       'Modus_Operandi',
        'Premise_Description',
       'Weapon_Description', 'Status_Description','Cross_Street'], axis=1)
Y_train = train[['Crime_Category']]

In [8]:
X_train.shape, X_test.shape

((20000, 13), (5000, 13))

In [9]:
new_categories = set(test['Victim_Descent'].unique()) - set(train['Victim_Descent'].unique())

In [10]:
#X_test=X_test.loc[~X_test['Victim_Descent'].isin(new_categories)]

In [11]:
#X_test=X_test.loc[~X_test['Victim_Descent'].isin(new_categories)]

In [12]:
X_test['Victim_Descent'] = X_test['Victim_Descent'].replace(to_replace=new_categories, value=np.nan)

In [13]:
'''new_categories = set(test['Cross_Street'].unique()) - set(train['Cross_Street'].unique())
X_test['Cross_Street'] = X_test['Cross_Street'].replace(to_replace=new_categories, value=np.nan)'''

"new_categories = set(test['Cross_Street'].unique()) - set(train['Cross_Street'].unique())\nX_test['Cross_Street'] = X_test['Cross_Street'].replace(to_replace=new_categories, value=np.nan)"

In [14]:
X_train.shape, X_test.shape

((20000, 13), (5000, 13))

In [15]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

'''label_cross_street  = LabelEncoder()
X_train['Cross_Street'].fillna("missing")
X_test['Cross_Street'].fillna("missing")
label_cross_street  = LabelEncoder()
X_train['Cross_Street'] = label_cross_street.fit_transform(X_train['Cross_Street'])
X_test['Cross_Street'] = label_cross_street.transform(X_test['Cross_Street'])'''

label_status  = LabelEncoder()
X_train['Status'].fillna("missing")
X_test['Status'].fillna("missing")
label_cross_street  = LabelEncoder()
X_train['Status'] = label_status.fit_transform(X_train['Status'])
X_test['Status'] = label_status.transform(X_test['Status'])

label_victim_descent  = LabelEncoder()
X_train['Victim_Descent'].fillna("missing")
X_test['Victim_Descent'].fillna("missing")
label_cross_street  = LabelEncoder()
X_train['Victim_Descent'] = label_victim_descent.fit_transform(X_train['Victim_Descent'])
X_test['Victim_Descent'] = label_victim_descent.transform(X_test['Victim_Descent'])

label_victim_sex  = LabelEncoder()
X_train['Victim_Sex'].fillna("missing")
X_test['Victim_Sex'].fillna("missing")
label_cross_street  = LabelEncoder()
X_train['Victim_Sex'] = label_status.fit_transform(X_train['Victim_Sex'])
X_test['Victim_Sex'] = label_status.transform(X_test['Victim_Sex'])

In [16]:
from sklearn.impute import SimpleImputer
value = 99999
# Specify the strategy as 'constant' and the fill_value as your desired static value
imputer = SimpleImputer(strategy='constant', fill_value=value)


In [17]:
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [18]:
import xgboost as xg
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import Binarizer
#### Calculating Sample Weights
n_samples, n_labels = Y_train.shape
from collections import Counter
# Calculate sample weights for each label column
sample_weights = []

for label_idx,name in zip(range(n_labels), Y_train.columns):
    
    label_col = Y_train.values[:, label_idx]
    print(name, end = ' ')
    
    
    class_counts = Counter(label_col)
    print(class_counts, end = '  ')
    
    total_instances = sum(class_counts.values())
    
    class_weights = {cls: total_instances / (count * len(class_counts)) for cls, count in class_counts.items()}
    print(class_weights)
    
    label_weights = [class_weights[cls] for cls in label_col]
    sample_weights.append(label_weights)
    
sum(label_weights)/len(label_weights)
np.array(sample_weights).T.shape
# Combine the sample weights for all label columns
sample_weights = np.array(sample_weights).T
sample_weights = np.mean(sample_weights, axis=1)
sample_weights.shape

Crime_Category Counter({0: 11666, 1: 4767, 2: 1808, 3: 1355, 4: 225, 5: 179})  {0: 0.2857306131778959, 1: 0.6992518005733864, 5: 18.6219739292365, 2: 1.8436578171091444, 3: 2.4600246002460024, 4: 14.814814814814815}


(20000,)

In [19]:
# Convert Crime_Category to category type
Y_train['Crime_Category'] = Y_train['Crime_Category'].astype('category')
# Encode the categorical column to numeric values
Y_train['Crime_Category'] = Y_train['Crime_Category'].cat.codes
print(Y_train.dtypes)

Crime_Category    int8
dtype: object


/tmp/ipykernel_33/4014897255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train['Crime_Category'] = Y_train['Crime_Category'].astype('category')
/tmp/ipykernel_33/4014897255.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train['Crime_Category'] = Y_train['Crime_Category'].cat.codes


In [20]:
X_test.shape

(5000, 13)

In [21]:
X_test

array([[ 3.401280e+01, -1.183045e+02,  2.000000e+03, ...,  4.000000e+02,
         2.000000e+00,  0.000000e+00],
       [ 3.407260e+01, -1.183029e+02,  1.700000e+03, ...,  9.999900e+04,
         2.000000e+00,  5.328000e+04],
       [ 3.393480e+01, -1.182695e+02,  9.000000e+02, ...,  9.999900e+04,
         2.000000e+00,  1.440000e+03],
       ...,
       [ 3.423920e+01, -1.184698e+02,  2.100000e+03, ...,  4.000000e+02,
         0.000000e+00,  0.000000e+00],
       [ 3.404530e+01, -1.182443e+02,  2.335000e+03, ...,  5.000000e+02,
         1.000000e+00,  1.440000e+03],
       [ 3.394380e+01, -1.182695e+02,  3.200000e+02, ...,  9.999900e+04,
         2.000000e+00,  0.000000e+00]])

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
import xgboost as xgb 
from sklearn.multiclass import OneVsOneClassifier, OneVsOneClassifier

xgb_model = xgb.XGBClassifier(max_depth=7, n_estimators=200, min_child_weight=3, gamma=0.5, subsample=1.0, colsample_bytree=1.0, scale_pos_weight=1, learning_rate=0.1,objective='multi:softprob')
xgb_model.fit(X_train, Y_train)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [13:44:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [24]:
preds = xgb_model.predict(X_test)
preds = pd.DataFrame(preds, columns=['Crime_Category'])

In [25]:
preds

,Crime_Category
0,1
1,0
2,3
3,0
4,2
...,...
4995,0
4996,0
4997,1
4998,1


In [26]:
preds.shape

(5000, 1)

In [27]:
'''from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, Y_train)'''

'from sklearn.ensemble import RandomForestClassifier\nclassifier = RandomForestClassifier(n_estimators=100, random_state=42)\nclassifier.fit(X_train, Y_train)'

In [28]:
'''preds = classifier.predict(X_test)
preds = pd.DataFrame(preds, columns=['Crime_Category'])'''

"preds = classifier.predict(X_test)\npreds = pd.DataFrame(preds, columns=['Crime_Category'])"

In [29]:
'''from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier(n_estimators=300)
classifier.fit(X_train, Y_train)

from sklearn.metrics import classification_report
preds = classifier.predict(X_test)

preds = pd.DataFrame(preds, columns=['Crime_Category'])'''

"from sklearn.ensemble import GradientBoostingClassifier\n\nclassifier = GradientBoostingClassifier(n_estimators=300)\nclassifier.fit(X_train, Y_train)\n\nfrom sklearn.metrics import classification_report\npreds = classifier.predict(X_test)\n\npreds = pd.DataFrame(preds, columns=['Crime_Category'])"

In [30]:

replacement_dict = {0:  'Property Crimes',
                    1 : 'Violent Crimes',
                    2 : 'Crimes against Public Order',
                    3 : 'Fraud and White-Collar Crimes',
                    4 : 'Crimes against Persons',  # Add placeholders for missing values (if needed)
                    5 : 'Other Crimes'} 
preds['Replaced_Crime_Category'] = preds['Crime_Category'].replace(replacement_dict)



In [31]:
preds

,Crime_Category,Replaced_Crime_Category
0,1,Violent Crimes
1,0,Property Crimes
2,3,Fraud and White-Collar Crimes
3,0,Property Crimes
4,2,Crimes against Public Order
...,...,...
4995,0,Property Crimes
4996,0,Property Crimes
4997,1,Violent Crimes
4998,1,Violent Crimes


In [32]:
#preds are my final output.
submission = pd.DataFrame({'ID':range(1,5001), 'Crime_Category': preds['Replaced_Crime_Category']})
submission.to_csv("submission.csv", index=False)

In [33]:
submission

,ID,Crime_Category
0,1,Violent Crimes
1,2,Property Crimes
2,3,Fraud and White-Collar Crimes
3,4,Property Crimes
4,5,Crimes against Public Order
...,...,...
4995,4996,Property Crimes
4996,4997,Property Crimes
4997,4998,Violent Crimes
4998,4999,Violent Crimes
